In [1]:
import urllib
import urllib.request
import urllib.parse
import bs4
import re
import os
import time
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import numpy as np
import konlpy.tag
import nltk
import json
#import pandas as pd
#import numpy as np
from konlpy.tag import Komoran
from konlpy.tag import Twitter
from time import time
import pickle
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
#import re

In [2]:
## 크롤링부분
def deleteTag(x):
    return re.sub("<[^>]*>", "", x)
 
def getComments(code):
    def makeArgs(code, page):
        params = {
            'code': code,
            'type': 'after',
            'isActualPointWriteExecute': 'false',
            'isMileageSubscriptionAlready': 'false',
            'isMileageSubscriptionReject': 'false',
            'page': page
        }
        return urllib.parse.urlencode(params)
 
    def innerHTML(s, sl=0):
        ret = ''
        for i in s.contents[sl:]:
            if i is str:
                ret += i.strip()
            else:
                ret += str(i)
        return ret
#
    def fText(s):
        if len(s): return innerHTML(s[0]).strip()
        return ''
# 
    retList = []
    colSet = set()
    print("Processing: %d" % code)
    page = 1
    while 1:
        try:
            f = urllib.request.urlopen(
                "http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?" + makeArgs(code, page))
            data = f.read().decode('utf-8')
        except:
            break
        soup = bs4.BeautifulSoup(re.sub("&#(?![0-9])", "", data), "html.parser")
        cs = soup.select(".score_result li")
        if not len(cs): break
        for link in cs:
            try:
                url = link.select('.score_reple em a')[0].get('onclick')
            except:
                print(page)
                print(data)
                raise ""
            m = re.search('[0-9]+', url)
            if m:
                url = m.group(0)
            else:
                url = ''
            if url in colSet: return retList
            colSet.add(url)
            cat = fText(link.select('.star_score em'))
            cont = fText(link.select('.score_reple p'))
            cont = re.sub('<span [^>]+>.+?</span>', '', cont)
            retList.append((url, cat, cont))
        page += 1
# 
    return retList
 
def fetch(i):
    outname = 'comments/%d.txt' % i
    try:
        if os.stat(outname).st_size > 0: return
    except:
        None
    rs = getComments(i)
    if not len(rs): return
    f = open(outname, 'w', encoding='utf-8')
    #f.write('INSERT IGNORE INTO movie VALUES ')
    for idx, r in enumerate(rs):
        if idx: f.write('\n')
        f.write("%d\t%s\t%s\t'%s'" % (i, r[0], r[1], r[2].replace("'", "''").replace("\\", "\\\\")))
    #f.write(';\n')
    f.close()
    time.sleep(1)

In [6]:
## 2018년 개봉 영화들의 영화 Code와 장르 가져오는 코드
def get2018(endpage = 51):
    tmp2 = []
    for i in range(1,endpage):
        URL = 'https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?open=2018&page='
        source_code_from_URL = urllib.request.urlopen(URL+str(i))
        soup = bs4.BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
        tmp = soup.find('ul', attrs={'class':'directory_list'}).find_all('li',recursive = False)
        
        
        for lst in tmp:
            
            genre = None
            for i in lst.findAll('a'):
                if 'genre' in i.get('href'):
                    genre  = i.text            
            
            tmp2.append([lst.find('a').get('href').strip('/movie/bi/mi/basic.nhn?code='),genre])
            
    return(tmp2)

In [7]:
## 2018년 개봉영화의 영화 Code와 장르 저장
MvDf = pd.DataFrame(get2018())
MvDf.columns = ['MovieNo','Genre']
MvDf.to_csv('MvDf.csv')

In [3]:
#MvDf = pd.read_csv('MvDf.csv')
## 한줄평 크롤링 실행
with ThreadPoolExecutor(max_workers=5) as executor:
    for movieno in MvDf['MovieNo']:
        executor.submit(fetch, int(movieno))

Processing: 178037
Processing: 153297
Processing: 164150
Processing: 162876
Processing: 168037
Processing: 160985
Processing: 149504
Processing: 137688
Processing: 144504
Processing: 152267
Processing: 144183
Processing: 171410
Processing: 171414
Processing: 154598
Processing: 178857
Processing: 85578
Processing: 149287
Processing: 162153
Processing: 166092
Processing: 173460
Processing: 174399
Processing: 143388
Processing: 175274
Processing: 179200
Processing: 150097
Processing: 170672
Processing: 172506
Processing: 162098
Processing: 161868
Processing: 170669
Processing: 173480
Processing: 169505
Processing: 172518
Processing: 178815
Processing: 162722
Processing: 172005
Processing: 174269
Processing: 163850
Processing: 170707
Processing: 172454
Processing: 158112
Processing: 168023
Processing: 153936
Processing: 153687
Processing: 174013
Processing: 157011
Processing: 170638
Processing: 172819
Processing: 164192
Processing: 164153
Processing: 166674
Processing: 154295
Processing: 1

Processing: 158890
Processing: 175365
Processing: 169572
Processing: 169548
Processing: 168115
Processing: 164173
Processing: 176907
Processing: 174808
Processing: 165026
Processing: 143373
Processing: 146736
Processing: 130756
Processing: 167099
Processing: 176994
Processing: 172061
Processing: 171887
Processing: 172561
Processing: 159636
Processing: 104390
Processing: 159894
Processing: 163008
Processing: 157974
Processing: 177352
Processing: 169552
Processing: 180380
Processing: 144216
Processing: 169648
Processing: 172010
Processing: 143401
Processing: 159833
Processing: 168735
Processing: 170684
Processing: 174062
Processing: 173498
Processing: 162072
Processing: 162148
Processing: 137926
Processing: 168159
Processing: 140799
Processing: 140064
Processing: 159645
Processing: 144130
Processing: 171413
Processing: 169551
Processing: 172425
Processing: 174387
Processing: 169568
Processing: 163681
Processing: 174034
Processing: 174835
Processing: 169530
Processing: 167049
Processing: 

Processing: 168011
Processing: 171764
Processing: 171746
Processing: 178816
Processing: 168047
Processing: 173504
Processing: 168036
Processing: 179687
Processing: 159827
Processing: 171466
Processing: 165450
Processing: 169688
Processing: 152694
Processing: 175268
Processing: 177335
Processing: 165348
Processing: 171801
Processing: 105297
Processing: 168742
Processing: 159892
Processing: 178654
Processing: 134906
Processing: 171879
Processing: 171765
Processing: 172562
Processing: 172469
Processing: 174878
Processing: 173020
Processing: 171436
Processing: 164269
Processing: 178404
Processing: 160699
Processing: 179138
Processing: 174447
Processing: 154653
Processing: 158601
Processing: 73087
Processing: 148026
Processing: 164183
Processing: 162395
Processing: 142726
Processing: 180361
Processing: 162423
Processing: 175263
Processing: 163824
Processing: 144585
Processing: 162367
Processing: 179699
Processing: 156195
Processing: 174099
Processing: 167747
Processing: 162715
Processing: 1